In [1]:
import openai
from openai import OpenAI
import ollama
import google.generativeai as genai
import os

# client = OpenAI(api_key="OPENAPI_API_KEY")
# genai.configure(api_key="GEMINI_API_KEY")

In [37]:
# origin, city1, city2, city3, d1, d2, d3, c1, c2, c3, t1, t2, t3 = 'kendrapada', 'cuttack', 'bhubaneswar', 'puri', 7, 5, 5, 'kendrapada and cuttack', 'cuttack and bhubaneswar', 'bhubaneswar and puri', '1', '2', '2'
origin = "delhi"
city1 = "kolkata"
city2 = "bengaluru"
city3 = "hyderabad"

d1 = 5
d2 = 7
d3 = 3

c1 = "delhi and kolkata"
c2 = "hyderabad and bengaluru"
c3 = "hyderabad and kolkata"

t1 = "10"
t2 = "4"
t3 = "5"

In [65]:
def generate_prompt(origin, city1, city2, city3, duration1, duration2, duration3, con1, con2, con3, t1, t2, t3):
    if con1:
        con1 = con1+"("+t1+" hours)"
    if con2:
        con2 = con2+"("+t2+" hours)" 
    if con3:
        con3 = con3+"("+t3+" hours)"
    connections = ", ".join((con1, con2, con3))
    
    total_duration = int(duration1) + int(duration2) + int(duration3)
    prompt = (f"You plan to visit 3 Indian cities for {total_duration} days in total. "
            f"You only take direct trains to commute between cities"
            f"You would like to visit {city1} for {duration1} days. "
            f"You would like to visit {city2} for {duration2} days, "
            f"You would like to visit {city3} for {duration3} days. "
            f"Here are the cities that have direct trains: {connections}. "
            f"You start the trip from {origin}"
            f"Find a trip plan of visiting the cities for {total_duration} days by taking direct trains to commute between them. "
            f"Do not generate aditional explanations. "
            f"Generate the output in the following manner satisfying the conditions given. You can modify the response if conditions are not met (d1, d2 .. are days, city1, city2 .. are cities, replace with actual days and cities"
            "Day d1 : travel from {city1} to {city2}. Spend d days. \n"
            "Day d2 : travel from {city2} to {city3}. Spend d days. \n"
            "Day d3 : travel from {city3} to {city4}. Spend d days. \n"
            "Day d4 : travel from {city4} to {city1}. Spend d days. "
            )
    prompt = (
            f"You plan to visit 3 Indian cities for a total of {total_duration} days. "
            f"You will only travel between cities using direct trains. "
            f"Your itinerary is as follows:\n"
            f"1. Visit {city1} for {duration1} days.\n"
            f"2. Visit {city2} for {duration2} days.\n"
            f"3. Visit {city3} for {duration3} days.\n"
            f"The cities with direct train connections are: {connections}. "
            f"Your trip starts in {origin}.\n"
            f"Create a travel plan that fits the conditions specified, focusing solely on the itinerary. "
            f"Do not include any additional explanations.\n"
            f"Format the output like this (replace placeholders with actual days and cities):\n"
            f"Day d1 : Travel from origin to city1. Visit city1 d days\n"
            f"Day d2 : Travel from city1 to city2. Visit city2 for d days\n"
            f"Day d3 : Travel from city2 to city3.  Visit city2 for d days\n"
            f"Day d4 : Travel from city3 to origin."
        )
    return prompt

In [67]:
prompt = generate_prompt(origin, city1, city2, city3, d1, d2, d3, c1, c2, c3, t1, t2, t3)
prompt

'You plan to visit 3 Indian cities for a total of 15 days. You will only travel between cities using direct trains. Your itinerary is as follows:\n1. Visit kolkata for 5 days.\n2. Visit bengaluru for 7 days.\n3. Visit hyderabad for 3 days.\nThe cities with direct train connections are: delhi and kolkata(10 hours), hyderabad and bengaluru(4 hours), hyderabad and kolkata(5 hours). Your trip starts in delhi.\nCreate a travel plan that fits the conditions specified, focusing solely on the itinerary. Do not include any additional explanations.\nFormat the output like this (replace placeholders with actual days and cities):\nDay d1 : Travel from origin to city1. Visit city1 d days\nDay d2 : Travel from city1 to city2. Visit city2 for d days\nDay d3 : Travel from city2 to city3.  Visit city2 for d days\nDay d4 : Travel from city3 to origin.'

In [69]:
response = ollama.chat(model='llama3.1:latest', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

In [70]:
print(response['message']['content'])

Day 1: Travel from Delhi to Kolkata. Visit Kolkata for 5 days
Day 6: Travel from Kolkata to Bengaluru. Visit Bengaluru for 7 days
Day 13: Travel from Bengaluru to Hyderabad. Visit Hyderabad for 3 days
Day 16: Travel from Hyderabad to Delhi


In [71]:
response = ollama.chat(model='mistral:latest', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

In [72]:
print(response['message']['content'])

 Day 1: Travel from Delhi to Kolkata (10 hours). Visit Kolkata for 5 days
   Day 6: Travel from Kolkata to Bengaluru (by train, duration unknown as direct trains may not be available, so consider adding layover time if needed). Visit Bengaluru for 7 days
   Day 13: Travel from Bengaluru to Hyderabad (4 hours). Visit Hyderabad for 3 days
   Day 16: Travel from Hyderabad to Delhi (5 hours).


In [73]:
response = ollama.chat(model='llama3.1:70b', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

In [74]:
print(response['message']['content'])

Here is the travel plan:

Day 1-5: Travel from Delhi to Kolkata (10 hours). Visit Kolkata for 5 days.
Day 6-7: Travel from Kolkata to Hyderabad (5 hours).
Day 8-14: Travel from Hyderabad to Bengaluru (4 hours). Visit Bengaluru for 7 days.
Day 15: Visit Hyderabad for 3 days and then travel back to Delhi is not possible due to the direct train connection constraint, however this can be modified as follows:
 
 Alternative solution considering only direct connections between cities:

 Day 1-5 : Travel from Delhi to Kolkata (10 hours). Visit Kolkata for 5 days.
Day 6: Travel from Kolkata to Hyderabad (5 hours).
Day 7-13: Travel from Hyderabad to Bengaluru is not possible in the same day, therefore visit Hyderabad first. 
Day 7-9: Visit Hyderabad for 3 days.
Day 10: Travel from Hyderabad to Bengaluru (4 hours).
Day 11-17: This would allow you to visit Bengaluru but exceeds the total number of days available. A modified version would be:
Day 10-15 and beyond: Visit Bengaluru for more than the

### GPT 4o mini

In [76]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [77]:
print(completion.choices[0].message.content)

Day 1 : Travel from Delhi to Kolkata. Visit Kolkata for 5 days  
Day 6 : Travel from Kolkata to Bengaluru. Visit Bengaluru for 7 days  
Day 13 : Travel from Bengaluru to Hyderabad. Visit Hyderabad for 3 days  
Day 15 : Travel from Hyderabad to Delhi.


### Gemini 1.5 flash

In [79]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)

In [80]:
print(response.parts[0].text)

Day 1 : Travel from Delhi to Kolkata. Visit Kolkata for 5 days.
Day 6 : Travel from Kolkata to Hyderabad. Visit Hyderabad for 3 days.
Day 9 : Travel from Hyderabad to Bengaluru. Visit Bengaluru for 7 days.
Day 16 : Travel from Bengaluru to Delhi. 

